In [31]:
from dash import Dash, html, callback, dcc, Output, Input, ctx
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random

df_rows = int(1e4)
df = pd.DataFrame()
df["A"] = [random.randint(1,10) for i in range(df_rows)]
df["B"] = [random.randint(1,10) for i in range(df_rows)]
df["C"] = [random.randint(1,10) for i in range(df_rows)]
df['label'] = random.choices(["Monday", "Tuedsay", "Wednesday"], k=df_rows)

In [32]:
def filter_data(df, value, slider):
    # subset data for vis
    _df = df[df.label == value] if value != None else df
    _df = _df[(_df.A <= int(slider)) &
                (_df.B <= slider) &
                (_df.C <= slider)]
    
    return _df

def run_summary_stats(df):
    df = df[["A", "B", "C"]].mean()
    return df

In [37]:
app = Dash(__name__)
app.layout = [
    html.H1(children="3D Shiznits", style={'textAlign':'left'}),
    dcc.Dropdown(df.label.unique(), 'Babagooey', id='dropdown-selection'),
    dcc.Slider(0, 10, .1, value=10, marks=None, 
              tooltip={"placement":"bottom", "always_visible":True},
              id='slider-value'),
    html.Button(id='btn-nclicks-1', n_clicks=0, children="Get Summary Stats", style={'margin-left': 10}),
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value'),
    Input('slider-value', 'value'),
    Input('btn-nclicks-1', 'n_clicks')
)
def update_graph(value, slider, button):
    #print(value, slider, button)

    # subset original dataframe
    _df = filter_data(df, value, slider)

    # plot it.
    raw_data = go.Scatter3d(x=_df.A, y=_df.B, z=_df.C,
                               marker=dict(opacity=0.025),
                               mode='markers',
                               name="data")
    traces = [raw_data]
    _title = f"{value} (data <= {slider})" if value != None else "Please select an option from the dropdown above"
    layout = go.Layout(title=_title, )

    # check if button count has changed
    if "btn-nclicks-1" == ctx.triggered_id:
        msg = "Button 1 was most recently clicked"
        #print("button -> ", value, slider)
        _obj = run_summary_stats(_df)

        summary_data = go.Scatter3d(x = [float(_obj.A)], y=[float(_obj.B)], z=[float(_obj.C)], 
                                    marker=dict(opacity=1),
                                    mode='markers',
                                   name="data mean")
        traces.append(summary_data)
        _title += f"<br>mean: {_obj.values}<br>std:    {_df[['A','B','C']].std().values}<br>"
        layout = go.Layout(title=_title, xaxis=dict(range=[0,10]), yaxis=dict(range=[0,10]))

    
    return go.Figure(data=traces, layout=layout)

In [38]:
app.run(debug=True)

In [39]:
app = Dash(__name__)
app.layout = [
    html.H1(children="3D Shiznits", style={'textAlign':'left'}),
    dcc.Dropdown(df.label.unique(), 'Babagooey', id='dropdown-selection'),
    dcc.Slider(0, 10, .1, value=10, marks=None, 
              tooltip={"placement":"bottom", "always_visible":True},
              id='slider-value'),
    html.Button(id='btn-nclicks-1', n_clicks=0, children="Get Summary Stats", style={'margin-left': 10}),
    html.Div([
            dcc.Graph(id='graph-content', style={'width': '120vh', 'height': '80vh'}),
            html.P(id='summary_stats'),
        ],
        style={ "height":"500px",
                "display": "flex"}),
    html.Br()
]

@callback(
    Output('graph-content', 'figure'),
    Output('summary_stats', 'children'),
    Input('dropdown-selection', 'value'),
    Input('slider-value', 'value'),
    Input('btn-nclicks-1', 'n_clicks')
)
def update_graph(value, slider, button):
    #print(value, slider, button)

    # subset original dataframe
    _df = filter_data(df, value, slider)

    # plot it.
    raw_data = go.Scatter3d(x=_df.A, y=_df.B, z=_df.C,
                               marker=dict(opacity=0.025),
                               mode='markers',
                               name="data")
    traces = [raw_data]
    _title = f"{value} (data <= {slider})" if value != None else "Please select an option from the dropdown above"
    data_display = ""
    layout = go.Layout(title=_title, )

    # check if button count has changed
    if "btn-nclicks-1" == ctx.triggered_id:
        msg = "Button 1 was most recently clicked"
        #print("button -> ", value, slider)
        _obj = run_summary_stats(_df)

        summary_data = go.Scatter3d(x = [float(_obj.A)], y=[float(_obj.B)], z=[float(_obj.C)], 
                                    marker=dict(opacity=1),
                                    mode='markers',
                                   name="data mean")
        traces.append(summary_data)
        #_title += f"<br>mean: {_obj.values}<br>std:    {_df[['A','B','C']].std().values}<br>"
        data_display = ["Data Summary", html.Br(), 
                        "* samples: ",  _df.shape[0], html.Br(),
                       "* mean: ",   str([float(round(i,2)) for i in _df[["A","B","C"]].mean().values]), html.Br(),
                       "* std dev:  ", str([float(round(i,2)) for i in _df[["A","B","C"]].std().values])]
        layout = go.Layout(title=_title, xaxis=dict(range=[0,10]), yaxis=dict(range=[0,10]))

    
    return [go.Figure(data=traces, layout=layout), data_display]

In [40]:
app.run(debug=True, host="127.0.0.1", port="8086")